<a href="https://colab.research.google.com/github/DavidSenseman/BIO1173/blob/main/Copy_of_Class_DICOM_V4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install Packages

In [ ]:
!pip install pydicom

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 88.1 MB/s eta 0:00:00


### **Download and Extract Data**

In [ ]:
# Example

from pathlib import Path
import requests
import zipfile
import sys
import shutil
import os
import warnings
import numpy as np
import pandas as pd
import pydicom
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data import DataLoader, TensorDataset, Dataset, random_split
from torchvision import transforms, models

# ------------------------------------------------------------------
# Configuration – change only if you want a different URL / filename
# ------------------------------------------------------------------
URL = "https://biologicslab.co/BIO1173/data/"
ZIP_FILENAME = "pna_data.zip"

# ------------------------------------------------------------------
# Download the zip file (streamed, so it works with large files)
# ------------------------------------------------------------------
def download_zip(url: str, dest: Path, chunk_size: int = 8192) -> None:
    """Download a file from `url` and write it to `dest`."""
    print(f"Downloading {ZIP_FILENAME} to {dest}...", end='')
    with requests.get(url, stream=True, timeout=30) as r:
        r.raise_for_status()           # will raise for 4xx/5xx
        with dest.open("wb") as f_out:
            for chunk in r.iter_content(chunk_size=chunk_size):
                if chunk:               # filter out keep‑alive new chunks
                    f_out.write(chunk)
    print("done")

# ------------------------------------------------------------------
# Un‑zip the downloaded archive into a *named* directory
# ------------------------------------------------------------------
def unzip_file(zip_path: Path, extract_to: Path) -> None:
    """Extract all members of `zip_path` into `extract_to`."""
    print(f"Unzipping {ZIP_FILENAME} to {extract_to}...", end='')
    with zipfile.ZipFile(zip_path, "r") as zf:
        zf.extractall(extract_to)
    print("done")

# ------------------------------------------------------------------
# Optional – delete the zip after extraction
# ------------------------------------------------------------------
def clean_up_zip(zip_path: Path) -> None:
    """Delete the zip file – only if you no longer need it."""
    zip_path.unlink()
    print(f"Removed temporary archive: {zip_path}... done")

# ------------------------------------------------------------------
# Main routine
# ------------------------------------------------------------------
def main() -> None:
    cwd          = Path.cwd()            # current working directory
    zip_path     = cwd / ZIP_FILENAME
    extract_dir  = cwd / zip_path.stem   # e.g. /pna_data

    # Ensure the extraction directory exists
    extract_dir.mkdir(parents=True, exist_ok=True)

    # Download
    download_zip(URL+ZIP_FILENAME, zip_path)

    # Un‑zip
    unzip_file(zip_path, extract_dir)

    # Clean‑up the downloaded archive
    clean_up_zip(zip_path)

    print(f"Files have been extracted to {extract_dir}")

# ------------------------------------------------------------------
if __name__ == "__main__":
    main()


Unzipping pna_data.zip to /content/pna_data...done
Removed temporary archive: /content/pna_data.zip... done
Files have been extracted to /content/pna_data


### **Process DICOM Images**



In [ ]:
# Example

import numpy as np
import pandas as pd
import pydicom
import warnings
import seaborn as sns

# Global settings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# -------------------------------------------------------------
#  Helper: read a single DICOM file
# -------------------------------------------------------------
def read_dicom_file(file_path: str):
    """Read a DICOM file and extract image data + basic metadata."""
    ds = pydicom.dcmread(file_path)

    # Basic metadata
    metadata = {
        'filename': os.path.basename(file_path),
        'patient_name': getattr(ds, 'PatientName', 'Unknown'),
        'patient_id': getattr(ds, 'PatientID', 'Unknown'),
        'study_date': getattr(ds, 'StudyDate', 'Unknown'),
        'study_time': getattr(ds, 'StudyTime', 'Unknown'),
        'modality': getattr(ds, 'Modality', 'Unknown'),
        'manufacturer': getattr(ds, 'Manufacturer', 'Unknown'),
        'institution_name': getattr(ds, 'InstitutionName', 'Unknown'),
        'series_description': getattr(ds, 'SeriesDescription', 'Unknown'),
        'bits_allocated': getattr(ds, 'BitsAllocated', 'Unknown'),
        'rows': getattr(ds, 'Rows', 'Unknown'),
        'columns': getattr(ds, 'Columns', 'Unknown'),
        'pixel_spacing': getattr(ds, 'PixelSpacing', 'Unknown')
    }

    # Image data
    if hasattr(ds, 'pixel_array'):
        image_array = ds.pixel_array

        # Normalise to 0‑255 if needed
        if image_array.dtype != np.uint8:
            image_array = ((image_array - image_array.min()) /
                           (image_array.max() - image_array.min()) * 255).astype(np.uint8)

        metadata['image_available'] = True
        metadata['image_shape'] = image_array.shape
    else:
        metadata['image_available'] = False
        metadata['image_shape'] = 'No image data'

    return ds, metadata

# -------------------------------------------------------------
#  Helper: fast drop‑check
# -------------------------------------------------------------
def is_file_dropped(file_path: str) -> bool:
    """
    Quick guard that tells us whether a DICOM file is already
    missing / unreadable.
    """
    if not os.path.isfile(file_path):
        return True

    if os.path.getsize(file_path) == 0:
        return True

    try:
        pydicom.dcmread(file_path, stop_before_pixels=True)
    except Exception:
        return True

    return False

# -------------------------------------------------------------
#  Load & preprocess – merge CSVs, keep only valid DICOM rows
# -------------------------------------------------------------
def load_and_preprocess_data(
    data_dir: str = '.',
    log_dropped: bool = True
):
    # Load the two CSVs, merge on patient ID, and keep only rows
    # that have an intact DICOM image.
    info_df   = pd.read_csv(os.path.join(data_dir, 'pna_detailed_class_info.csv'))
    labels_df = pd.read_csv(os.path.join(data_dir, 'pna_train_labels.csv'))

    # Define patient ID variable
    info_id_col   = 'patientId'
    labels_id_col = 'patientId'

    merged_df = pd.merge(info_df, labels_df, left_on=info_id_col,
                         right_on=labels_id_col, how='inner')

    dicom_dir = os.path.join(data_dir, 'pna_train_images')
    valid_rows = []
    dropped_ids = []

    for idx, row in merged_df.iterrows():
        patient_id = row[info_id_col]
        dicom_file = os.path.join(dicom_dir, f"{patient_id}.dcm")

        if is_file_dropped(dicom_file):
            dropped_ids.append(patient_id)
        else:
            valid_rows.append(idx)

    filtered_df = merged_df.loc[valid_rows].copy()
    print(f"Filtered DataFrame shape (with valid DICOM files): {filtered_df.shape}")

    return filtered_df, dropped_ids

# -------------------------------------------------------------
#  Build X, y arrays – skip any DICOM that fails
# -------------------------------------------------------------
def create_dataset(
    filtered_df: pd.DataFrame,
    data_dir: str = '.',
    target_column: str = 'Target',
    max_samples: int | None = None,
    verbose: bool = True,
    # optional: allow a reproducible random seed
    random_state: int | None = None,
) -> tuple[np.ndarray, np.ndarray]:
    """
    Build X, y arrays from the given DataFrame.

    Parameters
    ----------
    filtered_df : pd.DataFrame
        DataFrame that contains (at least) the columns `patientId` and the target.
    data_dir : str, default='.'
        Base directory that contains the sub‑folder `pna_train_images/`.
    target_column : str, default='Target'
        Column name that holds the target label.
    max_samples : int | None, default=None
        If provided, a random subset of this size will be chosen from
        ``filtered_df`` before loading.  If ``max_samples`` is larger
        than the number of available rows, all rows are used.
    verbose : bool, default=True
        Show progress / debug messages.
    random_state : int | None, default=None
        Seed for reproducible shuffling.  Pass an integer if you need
        deterministic behaviour; otherwise the selection is truly random.

    Returns
    -------
    X : np.ndarray
        Image data of shape (N, H, W, 3) with dtype uint8.
    y : np.ndarray
        Corresponding target labels of shape (N,) with dtype int.
    """
    import os
    from tqdm import tqdm
    import numpy as np

    if filtered_df.empty:
        if verbose:
            print("Input DataFrame is empty – nothing to load.")
        return np.array([], dtype=np.uint8), np.array([], dtype=int)

    # ---------- Randomly sub‑sample if requested ----------
    if max_samples is not None:
        n_available = len(filtered_df)
        if max_samples < n_available:
            # ``sample`` performs a random shuffle; replace=False guarantees
            # unique rows.  ``random_state`` makes the selection reproducible.
            filtered_df = filtered_df.sample(n=max_samples, random_state=random_state)
        else:
            if verbose:
                print(
                    f"Requested max_samples={max_samples} "
                    f"but only {n_available} rows are available – "
                    "using all rows."
                )
    # ---------- Build dataset ----------
    print("Creating dataset")

    dicom_dir = os.path.join(data_dir, "pna_train_images")

    iterator = tqdm(
        filtered_df.iterrows(),
        total=len(filtered_df),
        desc="Loading DICOM Images",
        disable=not verbose,
    )

    images, labels = [], []

    info_id_col, labels_id_col = 'patientId', 'patientId'

    for i, (_, row) in enumerate(iterator, start=1):
        patient_id = row[info_id_col]
        dicom_file = os.path.join(dicom_dir, f"{patient_id}.dcm")

        if not os.path.exists(dicom_file):
            if verbose:
                tqdm.write(f"[{i}] Missing file: {patient_id}")
            continue

        ds, metadata = read_dicom_file(dicom_file)

        if metadata is None or not metadata.get("image_available", False):
            if verbose:
                tqdm.write(f"[{i}] No image in {patient_id}")
            continue

        img = ds.pixel_array
        if img.ndim == 2:
            img = np.stack([img] * 3, axis=-1)

        images.append(img.astype(np.uint8))
        labels.append(int(row[target_column]))

    # ----------------------------------------------------------------------
    # Assemble the final arrays *after* the loop finishes
    # ----------------------------------------------------------------------

    X = np.stack(images, axis=0) if images else np.array([], dtype=np.uint8)
    y = np.array(labels, dtype=int) if labels else np.array([], dtype=int)

    if verbose:
        print(f"The number of Image files = {len(X)}")

    return X, y

# -------------------------------------------------------------
#  Main block – run the whole pipeline
# -------------------------------------------------------------
if __name__ == "__main__":
    # Path to the data root
    data_root = os.path.join('.', 'pna_data')

### **Generate Datasets**

In [ ]:
# Example

# Define number of images to use
MAX_SAMPLES=500

# Generate filtered_df
filtered_df, _ = load_and_preprocess_data(data_dir=data_root)

# Build X, y
X, y = create_dataset(filtered_df, data_dir=data_root, target_column='Target', max_samples=MAX_SAMPLES)

Filtered DataFrame shape (with valid DICOM files): (9337, 7)
Creating dataset


Loading DICOM Images: 100%|██████████| 500/500 [00:04<00:00, 123.60it/s]


The number of Image files = 500


### **Build Data Vectors**

In [ ]:
from typing import Tuple, Optional, Callable, Any

# Set Import Variables
NUM_EPOCHS: int=1
BATCH_SIZE: int=16
IMG_SIZE: int=244

# ------------------------------------------
# Helper: Build transforms
# ------------------------------------------
def get_transform(
    img_size=IMG_SIZE,
    is_train: bool = True,
    crop_size=IMG_SIZE,
    h_flip: bool = True,
    augment: bool = False
) -> Callable:
    """
    Returns a torchvision transform chain.
    """
    if is_train:
        transform = transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.RandomResizedCrop(crop_size) if augment else transforms.RandomCrop(crop_size),
            transforms.RandomHorizontalFlip() if h_flip else transforms.Lambda(lambda x: x),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),
        ])
    else:  # eval / test
        transform = transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.CenterCrop(crop_size),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),
        ])

    return transform


# ------------------------------------------
# Helper: Build dataloaders
# ------------------------------------------
def build_dataloaders(
    X: np.ndarray,
    y: np.ndarray,
    batch_size=BATCH_SIZE,
    val_split: float = 0.2,
    seed: int = 42,
    num_workers: int = 4,
) -> tuple[DataLoader, DataLoader]:
    """
    Returns training and validation DataLoaders.
    """
    dataset = DicomDataset(X, y, transform=get_transform(is_train=True))

    total = len(dataset)
    val_len = int(total * val_split)
    train_len = total - val_len

    torch.manual_seed(seed)
    train_ds, val_ds = random_split(dataset, [train_len, val_len])

    # Override transforms for validation
    val_ds.dataset.transform = get_transform(is_train=False)

    train_loader = DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True,
    )

    val_loader = DataLoader(
        val_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True,
    )

    return train_loader, val_loader

# ------------------------------------------
# Training loop
# ------------------------------------------
def train_one_epoch(
    model: nn.Module,
    loader: DataLoader,
    criterion: nn.Module,
    optimizer: optim.Optimizer,
    device: torch.device,
) -> float:
    model.train()
    epoch_loss = 0.0

    for imgs, targets in tqdm(loader, desc="Training", leave=False):
        imgs, targets = imgs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item() * imgs.size(0)

    return epoch_loss / len(loader.dataset)

# --------------------------------------------
# Measure validation loss during training
# --------------------------------------------
def validate(
    model: nn.Module,
    loader: DataLoader,
    criterion: nn.Module,
    device: torch.device,
) -> tuple[float, float]:
    model.eval()
    epoch_loss = 0.0
    correct = 0

    with torch.no_grad():
        for imgs, targets in tqdm(loader, desc="Validation", leave=False):
            imgs, targets = imgs.to(device), targets.to(device)

            outputs = model(imgs)
            loss = criterion(outputs, targets)

            epoch_loss += loss.item() * imgs.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == targets).sum().item()

    val_loss = epoch_loss / len(loader.dataset)
    val_acc = correct / len(loader.dataset)
    return val_loss, val_acc

# ------------------------------------------
# Get ResNet50 model
# ------------------------------------------
def get_resnet50(
    num_classes: int,
    pretrained: bool = True,
    device: torch.device | None = None,
    name: str | None = None
) -> nn.Module:
    """Return a ResNet‑50 backbone.  Optionally attach a `name` attribute."""
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    backbone = models.resnet50(pretrained=pretrained)
    backbone.fc = nn.Linear(backbone.fc.in_features, num_classes)
    backbone.to(device)

    # Attach a name only if one was supplied
    if name is not None:
        backbone.name = name
    return backbone

# ------------------------------------------
# Get ResNet101 model
# ------------------------------------------
def get_resnet101(
    num_classes: int,
    pretrained: bool = True,
    device: torch.device | None = None,
    name: str | None = None
) -> nn.Module:
    """Return a ResNet‑101 backbone.  Optionally attach a `name` attribute."""
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    backbone = models.resnet101(pretrained=pretrained)
    backbone.fc = nn.Linear(backbone.fc.in_features, num_classes)
    backbone.to(device)

    # Attach a name only if one was supplied
    if name is not None:
        backbone.name = name
    return backbone

# ------------------------------------------
# Plotting routine
# ------------------------------------------

def plot_training(history, num_epochs, batch_size):
    """
    Plots training/validation loss & accuracy
    """
    # ------------------------------------------------------------
    # Plot the curves
    # ------------------------------------------------------------
    epochs = np.arange(1, NUM_EPOCHS + 1)

    fig, ax1 = plt.subplots(figsize=(8, 5))
    ax2 = ax1.twinx()

    ax1.plot(epochs, history['train_loss'], label='train loss', color='C0')
    ax1.plot(epochs, history['val_loss'],  label='val loss',   color='C1')
    ax2.plot(epochs, history['val_acc'],   label='val acc',   color='C2')

    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax2.set_ylabel('Accuracy')
    ax1.set_ylim(bottom=0)
    ax2.set_ylim(bottom=0, top=1)

    # Merge legends from both axes
    lines, labels = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines + lines2, labels + labels2, loc='upper left')

    plt.title('Training on GPU – loss & accuracy')
    plt.tight_layout()
    return history

# ------------------------------------------
# Training routine
# ------------------------------------------
def run_training(
    X: np.ndarray,
    y: np.ndarray,
    num_epochs=NUM_EPOCHS,
    batch_size=BATCH_SIZE,
    lr: float = 1e-4,
    weight_decay: float = 1e-4,
    val_split: float = 0.2,
    device: torch.device | None = None,
    model: nn.Module | None = None
) -> dict:

    # -------------------------------------------------------------
    # Device handling
    # -------------------------------------------------------------
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # -------------------------------------------------------------
    # Prepare the dataset
    # -------------------------------------------------------------
    X = torch.tensor(X, dtype=torch.float32)
    y = torch.tensor(y, dtype=torch.long)

    # Split into train / val
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=val_split, stratify=y, random_state=42
    )

    train_ds = TensorDataset(X_train, y_train)
    val_ds   = TensorDataset(X_val,   y_val)

    batch_size = BATCH_SIZE
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, pin_memory=True)
    val_loader   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False, pin_memory=True)

    # -------------------------------------------------------------
    # Model handling - if no model provided, create one
    # -------------------------------------------------------------
    if model is None:
        # Create a ResNet50 model (since you have this function)
        print("WARNING: No model. Generating model")
        num_classes = int(y.max().item() + 1)  # Get number of classes
        model = get_resnet50(num_classes=num_classes, pretrained=True, device=device)

    model.to(device)

    # -------------------------------------------------------------
    # Loss, Optimiser, Scheduler
    # -------------------------------------------------------------
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(
        model.parameters(),
        lr=lr,
        weight_decay=weight_decay
    )
    scheduler = optim.lr_scheduler.CosineAnnealingLR(
        optimizer, T_max=num_epochs, eta_min=lr * 0.01
    )

    # -------------------------------------------------------------
    # Lists to store metrics
    # -------------------------------------------------------------
    train_losses, val_losses, val_accs = [], [], []

    # -------------------------------------------------------------
    # Epoch loop
    # -------------------------------------------------------------
    for epoch in range(1, NUM_EPOCHS + 1):
        # ---- Train ------------------------------------------------
        model.train()
        running_train_loss = 0.0
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)

            optimizer.zero_grad()
            logits = model(xb)
            loss   = criterion(logits, yb)
            loss.backward()
            optimizer.step()

            running_train_loss += loss.item() * xb.size(0)   # accumulate over batch

        epoch_train_loss = running_train_loss / len(train_loader.dataset)
        train_losses.append(epoch_train_loss)

        # ---- Validation ------------------------------------------
        model.eval()
        running_val_loss = 0.0
        correct = 0
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                logits = model(xb)
                loss   = criterion(logits, yb)

                running_val_loss += loss.item() * xb.size(0)

                preds = torch.argmax(logits, dim=1)
                correct += (preds == yb).sum().item()

        epoch_val_loss = running_val_loss / len(val_loader.dataset)
        epoch_val_acc  = correct / len(val_loader.dataset)

        val_losses.append(epoch_val_loss)
        val_accs.append(epoch_val_acc)

        # ---- Scheduler step ---------------------------------------
        scheduler.step()

        # ---- (Optional) print progress ----------------------------
        print(f"Epoch {epoch:02d}/{NUM_EPOCHS} | "
              f"train_loss={epoch_train_loss:.4f} | "
              f"val_loss={epoch_val_loss:.4f} | "
              f"val_acc={epoch_val_acc:.4f}")

    # -------------------------------------------------------------
    # Build and return the history dict
    # -------------------------------------------------------------
    history = {
        "train_loss": train_losses,
        "val_loss":   val_losses,
        "val_acc":    val_accs,
    }
    return history

### **Display Functions**

In [ ]:
# Example
from typing import Tuple, Optional, Callable, Any

# -------------------------------------------------------------
# Dataset wrapper
# -------------------------------------------------------------
class DicomDataset(Dataset):
    def __init__(
        self,
        X: np.ndarray,
        y: np.ndarray,
        transform: Optional[Callable] = None
    ):
        assert X.ndim == 4, f"X must be (N, H, W, C) but got shape {X.shape}"
        assert y.ndim == 1, f"y must be 1‑D but got shape {y.shape}"
        assert X.shape[0] == y.shape[0], "X and y must have the same number of samples"

        self.X = X
        self.y = y
        self.transform = transform

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        img = self.X[idx]          # shape (H, W, 3), dtype uint8
        label = int(self.y[idx])

        # Convert to PIL Image so torchvision transforms can work
        img = transforms.ToPILImage()(img)

        if self.transform:
            img = self.transform(img)

        return img, label

# -------------------------------------------------------------
#  Show a single DICOM image (for sanity checks)
# -------------------------------------------------------------
def display_dicom_image(file_path: str, figsize: tuple = (10, 10)):
    """Show a single DICOM image with proper orientation."""
    ds = pydicom.dcmread(file_path)
    if hasattr(ds, 'pixel_array'):
        img = ds.pixel_array
        if getattr(ds, 'PhotometricInterpretation', None) == 'MONOCHROME1':
            img = np.max(img) - img

        plt.figure(figsize=figsize)
        plt.imshow(img, cmap='gray')
        plt.title(ds.SOPClassUID)
        plt.axis('off')
        plt.show()
    else:
        print("This DICOM has no pixel data.")

# -------------------------------------------------------------
#  Visualise class / target distributions
# -------------------------------------------------------------
def visualize_data_distribution(filtered_df: pd.DataFrame):
    if filtered_df is None:
        return

    plt.figure(figsize=(12, 6))

    # Class distribution
    plt.subplot(1, 2, 1)
    if 'class' in filtered_df.columns:
        class_counts = filtered_df['class'].value_counts()
        class_counts.plot(kind='bar')
        plt.title('Class Distribution')
        plt.ylabel('Count')
    else:
        plt.text(0.5, 0.5, 'No class column found', ha='center')
        plt.title('Class Distribution')

    # Target distribution
    plt.subplot(1, 2, 2)
    if 'Target' in filtered_df.columns:
        target_counts = filtered_df['Target'].value_counts()
        target_counts.plot(kind='bar')
        plt.title('Target Distribution')
        plt.ylabel('Count')
    else:
        plt.text(0.5, 0.5, 'No Target column found', ha='center')
        plt.title('Target Distribution')

    plt.tight_layout()
    plt.show()

# -------------------------------------------------------------
#  Plot history
# -------------------------------------------------------------
def plot_history(
    history: dict,
    num_epochs: int,
    batch_size: int,
    title: str | None = None,
) -> dict:

    # ------------------------------------------------------------------
    # 1️⃣ Plot curves
    # ------------------------------------------------------------------
    epochs = np.arange(1, num_epochs + 1)

    fig, ax1 = plt.subplots(figsize=(8, 5))
    ax2 = ax1.twinx()

    ax1.plot(epochs, history["train_loss"], label="train loss", color="C0")
    ax1.plot(epochs, history["val_loss"],   label="val loss",   color="C1")
    ax2.plot(epochs, history["val_acc"],    label="val acc",    color="C2")

    ax1.set_xlabel("Epoch")
    ax1.set_ylabel("Loss")
    ax2.set_ylabel("Accuracy")
    ax1.set_ylim(bottom=0)
    ax2.set_ylim(bottom=0, top=1)

    # Merge legends from both axes
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc="upper left")

    # Optional custom title
    if title is None:
        title = "Training on GPU – loss & accuracy"
    plt.title(title)
    plt.tight_layout()

### **Train Model**

In [ ]:


import numpy as np
NUM_EPOCHS= 4

# Get model
NUM_CLASSES = len(set(y))          # y is your numpy array of labels
pna_model = get_resnet101(num_classes=NUM_CLASSES, name="pna_model")

# Train model
print(f"---Training is starting for {NUM_EPOCHS} epochs ----------")
history = run_training(
    X, y,
    num_epochs=NUM_EPOCHS,
    batch_size=BATCH_SIZE,
    lr=1e-4,
    weight_decay=1e-4,
    val_split=0.2
)

---Training is starting for 4 epochs ----------
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 225MB/s]


RuntimeError: Given groups=1, weight of size [64, 3, 7, 7], expected input[16, 1024, 1024, 3] to have 3 channels, but got 1024 channels instead